# Pixel Finder
## Finding Pixels you Want
### A more in-depth but slower searcher

This notebook takes a list of cubes (likely generated from the DatabaseSearcher) and goes though each cube looking for pixels that meet some kind of specific criteria. It then prints those pixels and a lot of data about said pixels to a file. 

The recorded pixel values are: 

0: cube number

1: flyby number

2: terrain type determined from mask

3: x coordinate in cube

4: y coordinate in cube

5: latitude

6: longitude

7: incidence angle

8: emission angle

9: azimuth angle

10: resolution in km

11: distance from pixel to a "no go" or "Null" location in the Mask. This value is present so we can dial how certain we want to be that certain areas are the terrain we say they are. 

12: phase angle

13: 0.93um I/F - There is one of these for each atmospheric window on Titan

14: 1.08um I/F

15: 1.27um I/F

16: 1.59um I/F

17: 2.01um I/F

18: 2.69um I/F

19: 2.79um I/F

20: 5.00um I/F

Currently, this notebook expects a .csv vile with flyby number and cube number on each entry, but nothing further is required. 



In [1]:
### PARAMETER SETTING
#This is where you tell the program what restrictions it should have on its observations.

inputFile = "equatorCubeList.csv" #Shoudl be a "...CubeList" file made by DatabaseSearcher. Includes flyby and cube number, nothing else. 
inputFile = "..\\DatabaseSearcher\\" + inputFile
outputName = "oppositionGenPixelResultsPt3.csv" #Best to put ".csv" at the end of the name. 
#IF YOU DO NOT CHANGE THAT IT WILL OVERRIDE THE FILE WITH THIS NAME

#Set values you want to check to True.
latLonCheck = False #Restricting to specific location ranges
flybyCheck = True #Temporal restriction by flyby number
resolutionCheck = False #Restrict to a certain resolution classificaiton.

maskDistCheck = False
maskDistResRelCheck = False #If you set both maskDistChecks to False, "Null" values will be accepted. 
#If all of these are set to False you're printing every single pixel, which will make a gigantic file most likely. Be careful.
#That said this IS what you want to do if you don't want to use the mask at all.
#If both are set to true, only pixels that satisfy both make it through.

#latLonCheck
#Bounds are not inclusive, this is to avoid edge case error values. (A point exactly at "0" is likely not real). 
latUpperBound = 30.0
latLowerBound = -30.0
lonUpperBound = 360.0
lonLowerBound = 0.0

#flybyCheck
flybyUpperBound = 200.0
flybyLowerBound = 26.0
#use 1 and 2 for TA and TB. 

#resolutionCheck
#Note: resolution 0 pixels are considered errors. 
resUpperBound = 50.0 #25.0 is considered standard. 
resLowerBound = 0.0

#Mask Distance Test
#This is the most unusual but perhaps the most helpful check: how close are we to a pixel we don't trust? 
#There are two ways to do this check: maskDistCheck (which is just a simple "less than X km" test) or maskDistResRelCheck (which compares the distance to an untrusted
#pixel to the resolution of the pixel itself. 

#maskDistcheck
maskDistLimit = 50.0
#Only distances greater than this (km) will be accepted.

#maskDistresRelCheck
maskDistRelLimit = 1.0/4.0 # 1/4 is the standard one, meaning the distance to an untrusted pixel must be four times the pixel's resolution to pass judgment. 

#Things that are NOT sorted here:
#Terrain Type. The ModelCreator does that. 
#Cube coordinates, since there's no reason to sift via those, they just help you locate the pixel again later on. 
#Viewing Geometry Angles. The models we create want the largest range of these you can obtain for proper tetrahedralization.
#I/F of various windows, as restricting these values will create inacurate models at the end. 
#Cube number, as the flyby number gives enough of a time judgment for our purposes. 

#Yes the only restrictive thing here that the DatabaseSearcher can't do is the use of the mask. So a standard run will only check against the mask, 
#assuming the DatabaseSearcher did a good enough job. However the options are still here if you want to slice the DatabaseSearcher's results
#into smaller pieces, which can happen regularly. 

oppositionCheck = False
#Wild and crazy new check: IS THE PIXEL WITHIN 5 DEGREES OF OPPOSITION? FIND OUT TODAY! 
#Opposition is just how close incidence and emission are to each other. We want them to be the same at 180 azimuth, more or less.
#math.acos(math.cos(INCI[j][i]*(np.pi/180))*math.cos(EMIS[j][i]*(np.pi/180))+math.sin(INCI[j][i]*(np.pi/180))*math.sin(EMIS[j][i]*(np.pi/180))*math.cos((180-AZIM[j][i])*(np.pi/180)))*(180/np.pi)


In [2]:
#REVISION FOR COLOR MULTI MASK. 

#Imports
import csv
import math
import numpy as np
import scipy.misc

#First, read in the data into a matrix. 
cubeList = []
with open(inputFile) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        triplet = ["A","A"]
        triplet[0] = row[0]
        triplet[1] = row[1]
        cubeList.append(triplet)
        line_count += 1

CLRMask = np.load("CLRMaskArray.npy")
#Need to make sure this file exists, user. 
#Colored Mask has all types of terrain marked, as well as no-go zones. 

CLRMaskDist = np.load("CLRMaskArrayDist.npy") #Holds the actual measures for distance at each point. 
#Now, for every item in cubeList, we find the file it points to and read it in. 
#Then we examine those files for *something*. Whatever it is. 
#Any pixel that matches that something, we keep. 

open(outputName, 'w').close() #Empty the file.
for item in cubeList:
    err = 0 #No error by default.

    flybyRangeAcceptable = False
    flyby = 0.
    if (item[0] == "TA"): flyby = 1.
    elif (item[0] == "TB"): flyby = 2.
    else: flyby = float(item[0][1:])
    if (flybyCheck == True): #Are we in the right flyby range?
        if (flybyLowerBound <= flyby and flybyUpperBound >= flyby):
            flybyRangeAcceptable = True
    else: 
        flybyRangeAcceptable = True #If we're not checking for time, all of them are fine. 


    if (flybyRangeAcceptable == True): #Don't bother reading anything in if we don't want this flyby. 
    
        #This code is copy adapted from the VIMS Cube Visualisation Interface Notebook. 
        #It is complicated.
        filepath = "C:\\Users\\deran\\Desktop\\CubeCSVDatabase\\" + item[0] + "\\CM_" + item[1] + ".cub.csv"
    
        #Now we extract the axes file as well...
        cubeAxesfp = filepath.removesuffix(".csv") + ".axes.csv"
        #and the geo files. 
        cubeGeofpIR = filepath.removesuffix(".cub.csv") + "_ir_geo.cub.csv"
        cubeGeofpIRaxes = filepath.removesuffix(".cub.csv") + "_ir_geo.cub.axes.csv"
    
        #Skeleton code nabbed from https://realpython.com/python-csv/
        
        #Step 1: use the axes to determine the size of what we're dealing with.
        xAxisCube = []
        yAxisCube = []
        zAxisCube = []
        
        xAxisGeoIR = []
        yAxisGeoIR = []
        zAxisGeoIR = []
    
        try:
            with open(cubeAxesfp) as csv_file: #remember to tab.
                    csv_reader = csv.reader(csv_file, delimiter=',')
                    line_count = 0
                    for row in csv_reader:
                        i = 0
                        L = len(row)
                        while (i < L-1):
                            if (line_count == 0):
                                xAxisCube.append(row[i])
                            elif (line_count == 1):
                                yAxisCube.append(row[i])
                            elif (line_count == 2):
                                zAxisCube.append(row[i])
                            i = i+1
                        line_count += 1
        except:
            print("No Cube Axes", item[0], item[1])
            err = 1 #whoops.        
        try:
            with open(cubeGeofpIRaxes) as csv_file: #remember to tab.
                    csv_reader = csv.reader(csv_file, delimiter=',')
                    line_count = 0
                    for row in csv_reader:
                        i = 0
                        L = len(row)
                        while (i < L-1):
                            if (line_count == 0):
                                xAxisGeoIR.append(row[i])
                            elif (line_count == 1):
                                yAxisGeoIR.append(row[i])
                            elif (line_count == 2):
                                zAxisGeoIR.append(row[i])
                            i = i+1
                        line_count += 1
        except:
            print("No Geo Axes", item[0], item[1])
            err = 1 #whoops.
            
        #We now have an x, y, and z axis. x and y axes are just ordinal, but the z axis contains wavelength in microns.
        #The lengths of these arrays tell us how to extract the data.
        
        cubeData = [[[0 for x in range(len(zAxisCube))] for x in range(len(yAxisCube))] for x in range(len(xAxisCube))]
        geoIRData = [[[0 for x in range(len(zAxisGeoIR))] for x in range(len(yAxisGeoIR))] for x in range(len(xAxisGeoIR))]
        
        #The above holds the data of the cube itself. 
        try:
            with open(filepath) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                line_count = 0
                i, j, k = 0, 0, 0
                for row in csv_reader:
                    while (i < len(xAxisCube)):
                        cubeData[i][j][k] = float(row[i])
                        if (math.isnan(cubeData[i][j][k])):
                            cubeData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                        elif (cubeData[i][j][k] < 0):
                            cubeData[i][j][k] = 0 #Negative values are nonsense.
                        elif (cubeData[i][j][k] > 1):
                            cubeData[i][j][k] = 1 #Make saturation obvious? Keep it from overloading. 
                        i = i + 1
                    i = 0
                    j = j + 1
                    if (j >= len(yAxisCube)):
                        j = 0
                        k = k + 1
                    line_count += 1
        except:
            print("No Cube File (how?)", item[0], item[1])
            err = 1 #whoops.
        try:
            with open(cubeGeofpIR) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                line_count = 0
                i, j, k = 0, 0, 0
                for row in csv_reader:
                    while (i < len(xAxisGeoIR)):
                        geoIRData[i][j][k] = float(row[i])
                        if (math.isnan(geoIRData[i][j][k])):
                            geoIRData[i][j][k] = 0 #We set nans to zero to allow plotting to take place, careful!
                        elif (geoIRData[i][j][k] < -1000):
                            geoIRData[i][j][k] = 0 #The default value is an extremely negative number. Scrub it.
                        i = i + 1
                    i = 0
                    j = j + 1
                    if (j >= len(yAxisGeoIR)):
                        j = 0
                        k = k + 1
                    line_count += 1
        except:
            print("No Geo File", item[0], item[1])
            err = 1 #Whoops.
        #The data is now read in.
    
        #BOOKKEEPING: declare where the windows are. 
        windowum = [0.933078, 1.08183, 1.27813, 1.59018, 2.01781, 2.69620, 2.79889, 5.00576]
        windowInd = [80, 108, 120, 139, 165, 206, 212, 344]
        windowIndAlt = [3,12,24,43,69,110,116,248]
        if (len(zAxisCube) <= 256):
            windowInd = windowIndAlt #Sometimes the cubes have different indeces. It's weird,  yeah, but this is how we check for that. 
    
        print("Working", item[0], item[1])
            
        #Now we can do stuff with the data. In this case, we need to examine every pixel and print out the "viable" ones to a file.
        if (err == 0):
            with open(outputName, 'a') as dataEntry:  
                x,y = 0,0 
                while (x < len(xAxisCube)):
                    y = 0
                    while (y < len(yAxisCube)):
                        #Inintial checks of pixel locaiton and resolution. Only continue if tests passed. 
                        if ((latLonCheck == False) or 
                            (geoIRData[x][y][0] > latLowerBound and geoIRData[x][y][0] < latUpperBound and 
                             geoIRData[x][y][1] > lonLowerBound and geoIRData[x][y][1] < lonUpperBound)): 
                            if((resolutionCheck == False) or 
                               (resUpperBound > geoIRData[x][y][2] and 
                                resLowerBound < geoIRData[x][y][2] and geoIRData[x][y][2] != 0.0)):
    
                                #Now at this point we perform the mask test, but we have to start gatehring some values before we can do that. 
        
                                lat = geoIRData[x][y][0]
                                lon = geoIRData[x][y][1]
                                #Round to the nearest whole number.
                                lat = int(np.rint(lat))
                                lon = 360 - int(np.rint(lon)) #Have to flip our longitude. 
                                #not because it's wrong, but because the INDEX of the MASK is FLIPPED.
                                if (lon >= 360): lon = 0
                                if (lat <= -89): lat = -89
                                #Now these are our latlon coordinates. We can use them to find the mask we need.
        
                                #But first, we have an issue. The distance to "bad pixel" is recorded in each pixel... but 
                                #How do we exctract it? Well first we need to find an existing value.
                                distMeasure = 1000 #A ridiculously oversized number.
                                if(CLRMask[90-lat][lon][0] != 0):
                                    distMeasure = CLRMaskDist[90-lat][lon][0]
                                elif(CLRMask[90-lat][lon][1] != 0):
                                    distMeasure = CLRMaskDist[90-lat][lon][1]
                                elif(CLRMask[90-lat][lon][2] != 0):
                                    distMeasure = CLRMaskDist[90-lat][lon][2]  
        
                                maskTest = False

                                if (maskDistCheck == False and maskDistResRelCheck == False):
                                    maskTest = True #Accept everything. 
                                else:
                                    if ((CLRMask[90-lat][lon][0] == 0 and CLRMask[90-lat][lon][1] == 0 and CLRMask[90-lat][lon][2] == 0)==False): #Black areas are no-go-zones, 
                                        #avoid them.
                                        if (maskDistResRelCheck == True):
                                            if ( (geoIRData[x][y][2] <= maskDistRelLimit*(distMeasure*100./255.)) and (geoIRData[x][y][2] != 0)): #0 value is probably an error, throw it out.
                                                maskTest = True
                                                if (maskDistCheck == True):
                                                    if ( (distMeasure*100./255.) < maskDistLimit ):
                                                        maskTest = False #If doing both tests, both must be true to be used.
                                        elif (maskDistCheck == True):
                                            if ( (distMeasure*100./255.) >= maskDistLimit ):
                                                maskTest = True 
                                        #The "else" case would be when neither is used, which we already accounted for.
                                
                                if (maskTest == True): #Do not bother with anything the mask hates.
                                    temp = np.transpose(cubeData)
                                    #Okay so guess what it works we have what we need time to actually GATHER ALL THE DATA.
                                    outfile = item[0] + "," #Cube number.
                                    outfile = outfile + item[1] + "," #Flyby number
                                    #outfile = outfile + item[2] + "," #Resolution classification 
        
                                    #Color Determination: what type of surface are we looking at? Label it!
                                    color = "Null" #No classification. Would be what a bad pixel recieved
                                    #But it should not be possible to get one at this point.
                                    if (CLRMask[90-lat][lon][0] == 0):
                                        if (CLRMask[90-lat][lon][1] == 0):
                                            if (CLRMask[90-lat][lon][2] == 0):
                                                color = "Null" #Black for no-go zone...
                                            else:
                                                color = "Lake" #Blue for Craters
                                        else:
                                            if (CLRMask[90-lat][lon][2] == 0):
                                                color = "Xanadu" #Green for Xanadu
                                            else:
                                                color = "Crater" #Cyan for Craters
                                    else:
                                        if (CLRMask[90-lat][lon][1] == 0):
                                            if (CLRMask[90-lat][lon][2] == 0):
                                                color = "Dunes" #Red for dunes
                                            else:
                                                color = "Labyrinth" #Magenta for Labyrinth
                                        else:
                                            if (CLRMask[90-lat][lon][2] == 0):
                                                color = "Hummocky" #Yellow for "hummocky" 
                                            else:
                                                color = "Plains" #White for Plains
        
                                    outfile = outfile + color + "," #Terrain type determined from mask color
                                    
                                    outfile = outfile + str(x) + "," + str(y) + "," #Pixel Coordinates
                                    outfile = outfile + str(geoIRData[x][y][0]) + "," + str(geoIRData[x][y][1]) + "," #Latlon
                                    #Fortunately we read lon in directly so the fact that we flipped it isn't an issue.
                                    inci = geoIRData[x][y][5]
                                    emis = geoIRData[x][y][6]
                                    azim = 0.
                                    #Azimuth formula from Jason Barnes' phasecurve.c++
                                    p = geoIRData[x][y][4] #used to calculate azimuth.
                                    ratio = -(np.cos(np.radians(p)) - np.cos(np.radians(inci))* np.cos(np.radians(emis)))/(np.sin(np.radians(inci))*np.sin(np.radians(emis)))
                                    azim = np.arccos(ratio)
                                    #Of course, this might be nan-ing. All the nans need their own values.
                                    if((math.isnan(azim) == True) and (ratio>0.)):
                                        azim = 0.
                                    elif((math.isnan(azim) == True) and (ratio<0.)):
                                        azim = 0.
                                    elif((inci==0) and (emis==0)):
                                        azim = 0.
                                    elif(math.isnan(azim) == True):
                                        print("Well you broke it, great. (Azimuth is nan, but could not be set to anything else.)")
                                    if (inci < 0):
                                        inci = 0
                                    elif (inci > 100):
                                        inci = 100
                                    if (emis < 0):
                                        emis = 0
                                    elif (emis > 90):
                                        emis = 90
                                    if (azim < 0):
                                        azim = 0
                                    if (azim > 180):
                                        azim = 180
                                    outfile = outfile + str(inci) + "," + str(emis) + "," #inci emis
                                    outfile = outfile + str(math.degrees(azim)) + "," #azimuth
                                    outfile = outfile + str(geoIRData[x][y][2]) + "," + str(distMeasure*100./255.) + "," + str(geoIRData[x][y][4]) + "," 
                                    #res distanceInMask phase
                                    #Now we need the window wavelengths, which means we need to *read* the data itself.
                                    outfile = outfile + str(np.transpose(temp[windowInd[0]])[x][y]) + "," 
                                    outfile = outfile + str(np.transpose(temp[windowInd[1]])[x][y]) + "," 
                                    outfile = outfile + str(np.transpose(temp[windowInd[2]])[x][y]) + "," 
                                    outfile = outfile + str(np.transpose(temp[windowInd[3]])[x][y]) + "," 
                                    outfile = outfile + str(np.transpose(temp[windowInd[4]])[x][y]) + "," 
                                    outfile = outfile + str(np.transpose(temp[windowInd[5]])[x][y]) + "," 
                                    outfile = outfile + str(np.transpose(temp[windowInd[6]])[x][y]) + ","
                                    outfile = outfile + str(np.transpose(temp[windowInd[7]])[x][y]) + "," + "\n"        
                                    #print(outfile)
                                    if (oppositionCheck == False):
                                        dataEntry.write(outfile)
                                    else:
                                        azim = 180-azim                                    
                                        if (math.acos(math.cos(inci*(np.pi/180))*math.cos(emis*(np.pi/180))+math.sin(inci*(np.pi/180))*math.sin(emis*(np.pi/180))*math.cos((180-azim)*(np.pi/180)))*(180/np.pi) < 5):
                                            dataEntry.write(outfile)
                                        
                        y=y+1
                    x=x+1 
    
#ADD SANITY CHECKER HERE IF NEEDED
    


C:\Users\deran\AppData\Local\Temp\ipykernel_24504\2004187583.py:7: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.3)
  import scipy.misc


Working T26 1552148542_1


C:\Users\deran\AppData\Local\Temp\ipykernel_24504\2004187583.py:270: RuntimeWarning: invalid value encountered in scalar divide
  ratio = -(np.cos(np.radians(p)) - np.cos(np.radians(inci))* np.cos(np.radians(emis)))/(np.sin(np.radians(inci))*np.sin(np.radians(emis)))


Working T26 1552197381_1
Working T26 1552198715_1
Working T27 1553553191_1


C:\Users\deran\AppData\Local\Temp\ipykernel_24504\2004187583.py:271: RuntimeWarning: invalid value encountered in arccos
  azim = np.arccos(ratio)


Working T28 1554942939_1
Working T28 1554945121_1
Working T28 1554945452_1
Working T28 1554945572_1
Working T31 1559075380_1
Working T31 1559077376_1
Working T31 1559077842_1
Working T31 1559078308_1
Working T31 1559079180_1
Working T31 1559079463_1
Working T31 1559079677_1
Working T31 1559079960_1
Working T32 1560456180_1
Working T33 1561834903_1
Working T33 1561835371_1


KeyboardInterrupt: 